# Setup & Import

In [1]:
!pip install python-terrier sentence-transformers faiss-gpu pandas

import pandas as pd
import pyterrier as pt
import os
import sys

# Setup Path
sys.path.append('../src')
from dense_retrieval import FaissRetriever # Import class yang kita buat tadi
from preprocessing import normalize_arabic # Kita pakai normalisasi saja, TANPA stemming

if not pt.started():
    pt.init()

DATA_PATH = '../data'
MODEL_PATH = '../models/fine_tuned_mbert' # Path model hasil fine-tuning
INDEX_PATH = '../data/indices/scenario_2_dense/faiss_index'

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for faiss-gpu


2026-02-14 22:48:32.986415: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-14 22:48:34.177175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771084114.821111   11378 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771084114.919896   11378 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771084115.320064   11378 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Training

In [2]:
# %% Cell 2: Training (Opsional di Notebook)
# Jalankan ini HANYA jika Anda belum punya model fine-tuned.
# Jika sudah run script dense_training.py terpisah, skip cell ini.

# from dense_training import train_dense_model
# TRIPLET_PATH = os.path.join(DATA_PATH, 'training/triplets.csv')
# if os.path.exists(TRIPLET_PATH):
#     train_dense_model(TRIPLET_PATH, output_path=MODEL_PATH)
# else:
#     print("File training tidak ditemukan, menggunakan model base (tanpa fine-tuning) untuk demo.")
#     MODEL_PATH = 'distilbert-base-multilingual-cased' # Fallback ke base model

# Indexing Dokumen Arab

In [3]:
# Load Dokumen Fiqih
df_docs = pd.read_csv(os.path.join(DATA_PATH, 'raw/fathul_muin.csv'))

# Preprocessing: HANYA Normalisasi (hapus harakat), TIDAK ADA Stemming di Skenario 2
# Sesuai spesifikasi: "Preprocessing: Tidak ada stemming... Menggunakan Sub-word Tokenization"
df_docs['text_clean'] = df_docs['text'].apply(normalize_arabic)

# Inisialisasi Retriever & Build Index
# Ini akan memakan waktu karena mengenkripsi seluruh teks Arab ke vektor
print("Building Dense Index...")
dense_retriever = FaissRetriever(
    model_path=MODEL_PATH,
    index_path=INDEX_PATH,
    doc_df=df_docs[['docno', 'text_clean']], # Gunakan teks yang sudah dinormalisasi ringan
    batch_size=32 # Sesuaikan dengan GPU Kaggle
)

KeyError: 'text'

# Retrieval Experiment

In [ ]:
# %% Cell 4: Retrieval Experiment
# Load Queries (Indo) & Qrels
df_queries = pd.read_csv(os.path.join(DATA_PATH, 'queries/queries_indo.csv'))
qrels = pd.read_csv(os.path.join(DATA_PATH, 'queries/qrels.csv'))

# Pastikan tipe data string
df_queries['qid'] = df_queries['qid'].astype(str)
qrels['qid'] = qrels['qid'].astype(str)
qrels['label'] = qrels['label'].astype(int)

# Jalankan Eksperimen PyTerrier
# dense_retriever sudah berupa pt.Transformer, jadi bisa langsung masuk pipeline
print("Running Dense Retrieval Experiment...")

pipeline = pt.Experiment(
    [dense_retriever],
    df_queries,
    qrels,
    eval_metrics=["map", "recip_rank", "recall_10", "recall_20"],
    names=["mBERT Dense Retrieval"]
)

print(pipeline)
# pipeline.to_csv('result_scenario_2.csv')